# Exploring dense convolution

This notebook uses the fibertree emulator to display the behaviour of various 1-D convolutions for dense data. Because the data is assumed to be dense we use the position-based operators on the premise that for dense data the position and coordinate are the same.

First, include some libraries

In [ ]:
# Begin - startup boilerplate code

import pkgutil

if 'fibertree_bootstrap' not in [pkg.name for pkg in pkgutil.iter_modules()]:
  !python3 -m pip  install git+https://github.com/Fibertree-project/fibertree-bootstrap --quiet

# End - startup boilerplate code

from fibertree_bootstrap import *

fibertree_bootstrap(style="uncompressed", animation="spacetime")

## Logging control

In [ ]:
# Initial values

enable_log = False

def set_params(log):
    global enable_log
    
    enable_log = (log == 'enable')

def log(*args):
    if enable_log:
        print(*args)

controls = interactive(set_params,
                       log=['disable', 'enable'])


display(controls)

## Create Input Tensors

Display controls to create the filter weights (**f**) and input activations (**i**)

In [ ]:
#
# Instantiate the tensor factory
#
tm = TensorMaker()

#
# Define the templates for two tensors
#
tm.addTensor(name="I",                    # required
             rank_ids=["W"],              # required
             shape=[8],                   # required
             density=1.0,                 # optional, default=0.2
             interval=5,                  # optional, default=5
             color="blue",                # optional, default="red"
             seed=10)                     # optional, default=10

tm.addTensor(name="F",                    # required
             rank_ids=["S"],              # required
                 shape=[3],               # required
                 density=1.0,             # optional, default=0.2
                 interval=5,              # optional, default=5
                 color="green",           # optional, default="red"
                 seed=10)                 # optional, default=10

#
# Display the controls to configure the tensors
#
tm.displayControls()

## Create the tensors

In [ ]:
i = tm.makeTensor("I")
f = tm.makeTensor("F")

W = i.getShape("W")
S = f.getShape("S")
Q = W - S + 1

print("Input activations")
displayTensor(i)
print("Filter Weights")
displayTensor(f)


## Verification Convolution

In [ ]:
i_raw = i.getRoot().uncompress()
f_raw = f.getRoot().uncompress()
o_verify_raw = Q*[0]

print(f"I: {i_raw}")
print(f"F: {f_raw}")

for q in range(Q):
    for s in range(S):
        w = q + s
        o_verify_raw[q] += i_raw[w] * f_raw[s]
        
print(f"O_verify: {o_verify_raw}")

o_verify = Tensor.fromUncompressed(name="O_verify",
                                   rank_ids=["Q"],
                                   shape=[Q],
                                   root=o_verify_raw,
                                   default=None)

displayTensor(o_verify)

## Output Stationary

$$ O_{q} = I_{q+s} \times F_s $$

In [ ]:
o = Tensor.fromUncompressed(rank_ids=["Q"],
                            shape=[Q],
                            root=Q*[0],
                            default=None,
                            name="o")

print("Convolution")

displayTensor(o)

canvas = createCanvas(f, i, o)

for q in range(Q):
    log(f"Processing output: ({q}, ({o[q]}))")
    for w in range(W):
        s = w - q
        if s < 0 or s >= S: continue
        log(f"  Processing input: ({w}, {i[w]})")
        log(f"  Processing filter weight ({s}, {f[s]})")
        o[q] += f[s] * i[w]

        window = [(w,) for w in range(q, q+S)]

        canvas.addActivity((), window, (), spacetime=("W", (q, w)))        
        canvas.addActivity((w-q,), (w,) , (q,), spacetime=("P", (q, w)))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Weight Stationary

In [ ]:
o = Tensor.fromUncompressed(rank_ids=["Q"],
                            shape=[Q],
                            root=Q*[0],
                            default=None,
                            name="o")

canvas = createCanvas(f, i, o)

print("Convolution")

for s in range(S):
    log(f"Processing weight: ({s}, {f[s]})")
    for q in range(Q):
        w = q+s
        log(f"  Processing input ({w}, {i[w]}")
        log(f"  Processing output ({q}, {o[q]})")
        
        o[q] += f[s] * i[w]
        
        canvas.addActivity((), [(w,) for w in range(q, q+S)], (), worker="W")
        canvas.addFrame((s,), (q+s,), (q,))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Input Stationary

In [ ]:
o = Tensor.fromUncompressed(rank_ids=["Q"],
                            shape=[Q],
                            root=Q*[0],
                            default=None,
                            name="o")

canvas = createCanvas(f, i, o)

print("Convolution")

for w in range(W):
    log(f"Processing input: ({w}, {i[w]})")
    for q in range(Q):
        s = w - q
        if s < 0 or s >= S: continue
        log(f"  Processing filter weight ({s}, {f[s]})")
        log(f"  Processing output ({q}, {o[q]}")
        o[q] += f[s] * i[w]

        f_window = [(w,) for w in range(q, q+S)]
        o_window = [(q,) for q in range(max(0,w-S+1),min(Q,w+1))]
        canvas.addActivity((), f_window, o_window, spacetime=("W", (w,q)))
        canvas.addActivity((w-q,), (w,), (q,), spacetime=("PE", (w,q)))


displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Output Stationary - Parallel Weight Processing

Assumes parallelism equal to the number of weights

In [ ]:
o = Tensor.fromUncompressed(rank_ids=["Q"],
                            shape=[Q],
                            root=Q*[0],
                            default=None,
                            name="o")


print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q in range(Q):
    log(f"Processing output: ({q}, {o[q]})")
    for w in range(W):
        s = w - q
        if s < 0 or s >= S: continue
        log(f"Processing input: ({w}, {i[w]})")
        log(f"  Processing filter weight ({s}, {f[s]})")
        o[q] += f[s] * i[w]

        canvas.addActivity((s,), (w,), (q,), spacetime=(s, q))

    f_window = [(w,) for w in range(q, q+S)]
    canvas.addActivity((),f_window, (), spacetime=("W", q))


displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Testing area

For running alternative algorithms